In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [65]:
def obtener_detalles_inmueble(url_inmueble):
    try:
        response = requests.get(url_inmueble)
        soup = BeautifulSoup(response.text, "html.parser")
        
        nombre = soup.find("h1", class_="ad-title")
        nombre = nombre.text.strip() if nombre else "Error al acceder al inmueble"
        
        precio = soup.find("div", class_="price__value jsPriceValue")
        precio = precio.text.strip() if precio else "Error al procesar precio"
        
        superficie = soup.find("span", class_="features__value")
        superficie = superficie.text.strip() if superficie else "Error al procesar superficie"
        
        actualizacion = soup.find("div", class_="details__block last-update")
        actualizacion = actualizacion.text.strip().replace("Última actualización\n", "").strip() if actualizacion else "Error al procesar actualizacion"
        
        # Características adicionales
        caracteristicas = soup.find("div", class_="features-container")
        detalles = {}
        if caracteristicas:
            secciones = caracteristicas.find_all("div", class_="features__content")
            for seccion in secciones:
                features = seccion.find_all("div", class_="features__feature")
                for feature in features:
                    label = feature.find("span", class_="features__label").text.strip().replace(":", "")
                    value = feature.find("span", class_="features__value").text.strip() if feature.find("span", class_="features__value") else "N/A"
                    detalles[label] = value
        
        return nombre, precio, superficie, actualizacion, detalles
    except Exception as e:
        return "Error al acceder al inmueble", "Error al acceder al inmueble", "Error al acceder al inmueble", "Error al acceder al inmueble", {}


In [66]:
def obtener_inmuebles_paginas(base_url, paginas=1):
    todos_los_inmuebles = []
    for i in range(1, paginas + 1):
        print(f"Procesando página {i}")
        url_pagina = f"{base_url}{i}/"
        response = requests.get(url_pagina)
        soup = BeautifulSoup(response.text, "html.parser")
        titulos_soup = soup.find_all("a", class_="ad-preview__title")
        for titulo_soup in titulos_soup:
            nombre = titulo_soup.text.strip()
            href = "https://www.pisos.com" + titulo_soup['href']
            nombre_detalle, precio, superficie, actualizacion, detalles = obtener_detalles_inmueble(href)
            inmueble = {
                "nombre": nombre,
                "precio": precio,
                "superficie": superficie,
                "href": href,
                "actualizacion": actualizacion
            }
            inmueble.update(detalles)
            todos_los_inmuebles.append(inmueble)
        time.sleep(2)
    df_alquileres = pd.DataFrame(todos_los_inmuebles)
    return df_alquileres

In [67]:
base_url = "https://www.pisos.com/alquiler/pisos-madrid/"
df_alquileres = obtener_inmuebles_paginas(base_url, paginas=1)

Procesando página 1


In [68]:
df_alquileres

,nombre,precio,superficie,href,actualizacion,Superficie construida,Habitaciones,Baños,Planta,Referencia,...,Garaje,Jardín,Vidrios dobles,Agua,Sistema de seguridad,Trastero,Gas,Chimenea,Interior,Teléfono
0,Piso en Zona Avenida de Europa,1.400 €/mes,65 m²,https://www.pisos.com/alquilar/piso-zona_aveni...,Anuncio actualizado el 13/02/2025,65 m²,1,1,2ª,IF5032-avda-1d-V T 40m,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Piso en Valdezarza,950 €/mes,50 m²,https://www.pisos.com/alquilar/piso-valdezarza...,Anuncio actualizado el 13/02/2025,50 m²,1,1,6ª,IF5032-J. Andrés,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Piso en calle de Serrano,3.900 €/mes,93 m²,https://www.pisos.com/alquilar/piso-recoletos2...,Anuncio actualizado el 13/02/2025,93 m²,2,2,NaN,IF5271-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apartamento en Avenida del Ferrol,1.200 €/mes,70 m²,https://www.pisos.com/alquilar/apartamento-pil...,Anuncio actualizado el 13/02/2025,70 m²,1,1,Bajo,513149-2016287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Loft en calle de Las Navas de Tolosa, 9",3.700 €,205 m²,https://www.pisos.com/alquilar/loft-sol_barrio...,Anuncio actualizado el 13/02/2025,205 m²,2,3,1ª,4527206-NAV9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Piso en calle de Poza de la Sal,1.400 €/mes,111 m²,https://www.pisos.com/alquilar/piso-villa_de_v...,Anuncio actualizado el 11/02/2025,111 m²,3,2,2ª,4400-23690,...,1,Comunitario,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Estudio en calle de San Florencio, 5",1.150 €,40 m²,https://www.pisos.com/alquilar/estudio-numanci...,Anuncio actualizado el 13/02/2025,40 m²,1,1,Bajo,4527206-SFL5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Piso en Aravaca,1.500 €/mes,90 m²,https://www.pisos.com/alquilar/piso-moncloa_ar...,Anuncio actualizado el 13/02/2025,90 m²,2,1,2ª,IF5032-Pza_2dvac,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Estudio en calle de Pollensa, 5",1.147 €/mes,50 m²,https://www.pisos.com/alquilar/estudio-parque_...,Anuncio actualizado el 05/02/2025,50 m²,NaN,1,NaN,IF5032-tartesos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Piso en calle de Felipe IV,3.500 €/mes,120 m²,https://www.pisos.com/alquilar/piso-jeronimos2...,Anuncio actualizado el 08/02/2025,120 m²,3,2,3ª,4400-23695,...,NaN,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
df_alquileres.shape


(31, 45)

In [70]:
df_alquileres.columns

Index(['nombre', 'precio', 'superficie', 'href', 'actualizacion',
       'Superficie construida', 'Habitaciones', 'Baños', 'Planta',
       'Referencia', 'Armarios empotrados', 'Ascensor', 'Piscina',
       'Calefacción', 'Antigüedad', 'Conservación', 'Amueblado', 'Tipo suelo',
       'Aire acondicionado', 'Cocina equipada', 'Balcón', 'Orientación',
       'Superficie útil', 'Exterior', 'Gastos de comunidad',
       'Carpintería interior', 'Carpintería exterior', 'Portero automático',
       'Puerta blindada', 'Adaptado a personas con movilidad reducida',
       'Terraza', 'Soleado', 'No se aceptan mascotas', 'Comedor', 'Lavadero',
       'Garaje', 'Jardín', 'Vidrios dobles', 'Agua', 'Sistema de seguridad',
       'Trastero', 'Gas', 'Chimenea', 'Interior', 'Teléfono'],
      dtype='object')